In [5]:
import transformers
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import re
import random
import datasets
from datasets import load_dataset
from abc import ABC, abstractmethod
from typing import List, Optional, Tuple, Dict
from tqdm import tqdm
import pickle
from dotenv import load_dotenv
import openai
import json

from typing import List, Optional, Tuple, Dict
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

import plotly.graph_objects as go
import plotly.express as px

from utils import untuple, eval_completions
from act_add.contrast_dataset import ContrastDataset
from scripts.get_activations import gen_pile_data, compare_token_lists, slice_acts
from act_add.model_wrapper import ModelWrapper
%load_ext autoreload
%autoreload 2

In [6]:
model_name = 'EleutherAI/pythia-12b'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side = 'left')
tokenizer.pad_token = tokenizer.eos_token

# dataset_name = 'duped.12b'
# N_PROMPTS = 5000
# mem_data = load_dataset('EleutherAI/pythia-memorized-evals')[dataset_name]
# pile_prompts = gen_pile_data(N_PROMPTS, tokenizer, min_n_toks = 64)
mw = ModelWrapper(model, tokenizer)

Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.75s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# load probe
from probes import LRProbe
path = '../../gld/train-data-probes/data/12b/'

# open .pth file
probe_weights = torch.load(path + 'probe_weights_regged.pth')
probe = LRProbe.from_weights(probe_weights['net.0.weight'], probe_weights['net.0.bias'])
probe.to('cuda')

LRProbe(
  (net): Sequential(
    (0): Linear(in_features=5120, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [8]:
from datasets import load_from_disk, Dataset, DatasetDict

dataset = load_from_disk(os.path.join(path, 'split_hf_token_dataset_vary_len_v2'))

# filter for labels == 1
prompts = [x[:32] for x in dataset['test'].filter(lambda x: x['labels'] == 1)['input_ids']]
prompts = tokenizer.batch_decode(prompts, skip_special_tokens=True)
prompts = list(set(prompts))

print(len(prompts))

1041


In [9]:
mw.rej_sampl_generate([prompts[0], prompts[1]], probe, 34, log_rej_samples=True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


part 0 of generation
[1. 1.]
Input ids tensor([[ 2112,   342,   436,   789,    28,   604,   417,    13,  3630,   281,
           253,  7648,  9107,  6807,    13,   187,   475,  3690,   904,  8319,
         17179,   659,    13, 17538, 32370,    13,  9693,    13,  6908,   470,
         49352,    14],
        [ 2625,   323, 42323,  2425,    15, 28292, 28706,   285, 24517,  9753,
          9526,  9218,    74,  9435, 34192, 10145,  3438, 17979,  3918,  9915,
         26656,  9915, 26656, 28292, 44946, 35200,    27,  5219,  3995,  4750,
           398,     0]], device='cuda:0')
Generation tensor([[ 2112,   342,   436,   789,    28,   604,   417,    13,  3630,   281,
           253,  7648,  9107,  6807,    13,   187,   475,  3690,   904,  8319,
         17179,   659,    13, 17538, 32370,    13,  9693,    13,  6908,   470,
         49352,    14, 39677,  5106,    15,   187,   475],
        [ 2625,   323, 42323,  2425,    15, 28292, 28706,   285, 24517,  9753,
          9526,  9218,    74,  9435

Exception: 